In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
total_costs = pd.ExcelFile('../data/Data - Consolidated.xlsx')

student_incomes = pd.ExcelFile('../data/mean-median-student-income.xlsx').parse('Columbia')
inflation_ratios = student_incomes[['year', 'inflation ratio']]

## Median US Income

In [3]:
# Median income in 2018 dollars for US
columnRenames = {
    'Year': 'year',
    'Median Income \n(in 2018 dollars)': 'income',
    'private (2018 dollars)': 'cost'
}
national_data_df = total_costs.parse('US Trends').rename(
    columns=columnRenames
)[columnRenames.values()]

In [4]:
national_data = []
for _, row in national_data_df.iterrows():
    d = {}
    for k in ['year', 'income', 'cost']:
        val = float(row[k]) if k == 'cost' else int(row[k])
        if np.isnan(val):
            continue
        d[k]= val
    national_data.append(d)
national_data = sorted(national_data, key=lambda d: d['year'])

## Get total costs for Ivies
**Right now unadjusted for inflation**

In [5]:
# The names of sheets in the excel file
ivies = ['Columbia', 'Brown', 'Cornell', 'Dartmouth', 'Harvard', 'Princeton', 'UPenn', 'Yale']

# Flatten the dataframes to: [ { school, year, cost } ]
school_costs = []
for ivy in ivies:
    sheet = total_costs.parse(ivy).rename(columns={'School Year (start)': 'year', 'School Year': 'year'})
    school_costs.append({
        'school': ivy,
        'data': sorted([
            {'year': int(row['year']), 'cost': int(row['total cost'])}
            for _, row in sheet[['year', 'total cost']].iterrows()
        ], key=lambda d: d['year'])
    })
'''
school_costs = {'series': []}
for ivy in ivies:
    sheet = total_costs.parse(ivy).rename(columns={'School Year (start)': 'year', 'School Year': 'year'})
    school_costs['years'] = [int(row['year'])
            for _, row in sheet[['year', 'total cost']].iterrows()]
    school_costs['series'].append({
        'school': ivy,
        'values': [int(row['total cost'])
            for _, row in sheet[['year', 'total cost']].iterrows()],
    })
'''
school_costs

[{'school': 'Columbia',
  'data': [{'year': 2003, 'cost': 40140},
   {'year': 2004, 'cost': 42598},
   {'year': 2005, 'cost': 44646},
   {'year': 2006, 'cost': 46874},
   {'year': 2007, 'cost': 49260},
   {'year': 2008, 'cost': 51406},
   {'year': 2009, 'cost': 54294},
   {'year': 2010, 'cost': 56681},
   {'year': 2011, 'cost': 59208},
   {'year': 2012, 'cost': 61540},
   {'year': 2013, 'cost': 64144},
   {'year': 2014, 'cost': 66604},
   {'year': 2015, 'cost': 69084},
   {'year': 2016, 'cost': 71785},
   {'year': 2017, 'cost': 74199},
   {'year': 2018, 'cost': 76856}]},
 {'school': 'Brown',
  'data': [{'year': 2003, 'cost': 39602},
   {'year': 2004, 'cost': 41538},
   {'year': 2005, 'cost': 44280},
   {'year': 2006, 'cost': 46340},
   {'year': 2007, 'cost': 48660},
   {'year': 2008, 'cost': 50560},
   {'year': 2009, 'cost': 52030},
   {'year': 2010, 'cost': 54370},
   {'year': 2011, 'cost': 56150},
   {'year': 2012, 'cost': 58140},
   {'year': 2013, 'cost': 60460},
   {'year': 2014, '

## Generate outputs

In [7]:
# writing outputs…
with open('../outputs/school-costs.json', 'w') as f:
    json.dump(school_costs, f)
with open('../outputs/us_incomes.json', 'w') as f:
    json.dump(national_data, f)

# …but also  directly writing to the recession finaid repo lol
with open('../../recession-financial-aid/data/school-costs.json', 'w') as f:
    json.dump(school_costs, f, indent=2)
with open('../../recession-financial-aid/data/national-data.json', 'w') as f:
    json.dump(national_data, f, indent=2)